In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.11274886131286621
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 23.04376653236073
200000 32.47234445274248
300000 32.97493054400857
400000 32.856624658023634
500000 32.35464904812144
600000 34.19132724118226
700000 31.952079778716104
800000 32.27440818643875
900000 32.290284901034326
1000000 31.266808271927815
1100000 13.703660715194179
1200000 28.333622083038232
1300000 31.76723461757459
1400000 31.884799686199784
1500000 30.95466714051429
1600000 32.44081337307516
1700000 32.587076074902235
1800000 31.75683571547648
1900000 31.683080600942812
2000000 31.388836465871126
2100000 32.075888957425946
2200000 8.231179372104755
2300000 36.2528163736346
2400000 39.6692173456846
2500000 37.35471700874447
2600000 39.33560243695015
2700000 37.46896202224511
2800000 37.79413213554135
2900000 37.26014103812213
3000000 35.78006618

29600000 30.21685654240752
29700000 32.95521236366584
29800000 27.49787661858813
29900000 17.122342365285018
30000000 30.908266731111237
30100000 31.53921925324191
30200000 33.62256377645019
30300000 20.18380341052808
30400000 24.21515886618978
30500000 32.732500856163355
30600000 31.37335727109809
30700000 34.31761837578287
30800000 29.890572956646693
30900000 31.830650434205676
31000000 29.46041607481468
31100000 26.201626370833466
31200000 13.7839589072914
31300000 27.78989740522934
31400000 30.734784237793114
31500000 29.875620543053785
31600000 29.404452153861648
31700000 32.485074417676174
31800000 29.494196798036285
31900000 32.3368208324515
32000000 12.697822073784877
32100000 30.584678517372787
32200000 32.717165137166525
32300000 30.069103089781482
32400000 32.37634664269582
32500000 11.337162851063301
32600000 30.302571792827827
32700000 32.23023055119484
32800000 29.159219211524125
32900000 30.354156377870467
33000000 28.06849201449965
33100000 27.006486811252827
33200000 2

59300000 24.629429643899858
59400000 24.026353671386346
59500000 25.912294545827844
59600000 25.762876602992453
59700000 11.890298614770112
59800000 27.939515458177365
59900000 27.46418898590271
60000000 28.909468270433127
60100000 27.36612916386087
60200000 28.611667466205073
60300000 28.593490776562522
60400000 27.797011746460324
60500000 28.573296314434227
60600000 15.345564836554036
60700000 22.47831595511804
60800000 25.941734283442855
60900000 25.344952770594748
61000000 25.85797855197084
61100000 24.64805192888936
61200000 12.726406357212698
61300000 21.91301401487001
61400000 24.832167158875958
61500000 25.031068536404646
61600000 24.340116042119135
61700000 25.441417166348437
61800000 15.087091862727243
61900000 23.63990686100286
62000000 27.937488407355477
62100000 26.92896360479341
62200000 27.972688080774052
62300000 27.676926114818457
62400000 28.745892621922835
62500000 27.370132684341208
62600000 28.2465537800327
62700000 27.76801532664859
62800000 10.24439343094353
6290

89000000 23.00229298041408
89100000 12.66691936778198
89200000 18.483230840017832
89300000 23.392914635939718
89400000 22.944427090208215
89500000 12.177308601914016
89600000 21.941509279439686
89700000 22.287388731263956
89800000 24.01430977931593
89900000 23.867336122137704
90000000 25.624669501545984
90100000 24.710358414507727
90200000 24.739886199412613
90300000 26.172472300869167
90400000 25.8604525054442
90500000 25.912674042713245
90600000 25.824639263181247
90700000 23.299431935979623
90800000 23.862631583967183
90900000 10.779209768269714
91000000 22.09869434184292
91100000 24.91758196782732
91200000 21.9740808352246
91300000 9.10325934446777
91400000 23.20914738241714
91500000 23.208458949943754
91600000 16.801993817607755
91700000 16.725512706188905
91800000 23.73008138585283
91900000 23.56270816821911
92000000 24.25097805539982
92100000 26.22592891581954
92200000 24.557896364810517
92300000 26.04618945184065
92400000 24.673028456083383
92500000 25.44480272326732
92600000 2

118100000 24.014006309068172
118200000 23.064895877107876
118300000 21.800338708741645
118400000 22.17060873259908
118500000 20.274429215349727
118600000 21.513710555876262
118700000 20.984246875264073
118800000 19.627998757997815
118900000 20.90001237986237
119000000 12.148312097826663
119100000 16.947562166015636
119200000 7.626923354993404
119300000 14.321399423682813
119400000 17.159945290441275
119500000 19.465266303229765
119600000 20.682299553741995
119700000 20.20848998811005
119800000 19.40866958311784
119900000 21.387701034547145
120000000 19.6514418497075
120100000 22.08863020052088
120200000 23.0442214347606
120300000 21.910120189861708
120400000 23.337297684295677
120500000 20.40546726788267
120600000 21.069611393509685
120700000 20.609443561444014
120800000 19.640079175993755
120900000 21.0187693379834
121000000 19.80998369381919
121100000 19.728943457473314
121200000 7.917993932977104
121300000 12.873048921574988
121400000 14.96257173413831
121500000 12.622787890911647
1

...

290200000 4.70815816210966
290300000 5.356711372389081
290400000 5.103022162532942
290500000 5.3354545110523155
290600000 5.238850812024218
290700000 4.802160932965821
290800000 3.694442419302236
290900000 1.1383238536838967
291000000 1.135331219282789
291100000 5.910488285828631
291200000 4.7909197815134945
291300000 5.254395845419938
291400000 5.271941393129938
291500000 5.008977403770971
291600000 4.965967387037662
291700000 4.885283160760139
291800000 5.3661930407914635
291900000 5.035701171228734
292000000 6.113871763380463
292100000 5.360388391343787
292200000 5.323788624677903
292300000 5.248892225592036
292400000 4.674198771272874
292500000 5.207001209026391
292600000 5.1484518038550116
292700000 5.197006505040581
292800000 4.773719619692267
292900000 5.199936976881092
293000000 1.098763382416292
293100000 1.106473896399364
293200000 4.034197563360916
293300000 4.742650442986339
293400000 4.910497191191218
293500000 5.084063059584515
293600000 4.81006886727314
293700000 4.90963

319300000 3.8117940302157876
319400000 2.908166918396234
319500000 3.2278814925084047
319600000 2.9042133218580335
319700000 3.319033154895765
319800000 3.442993325824525
319900000 4.074658734903523
320000000 4.084046433730443
320100000 3.700590840112337
320200000 3.2450069706388587
320300000 3.3887551878176447
320400000 3.394255014028804
320500000 3.408694263515638
320600000 3.82373383926181
320700000 3.1957793113652997
320800000 3.9654857244876585
320900000 1.0848440400421024
321000000 0.6362875128107229
321100000 1.7476435130268289
321200000 3.9869044646409213
321300000 2.05981971502423
321400000 3.460990836212682
321500000 3.2864540255871755
321600000 3.0157572607229337
321700000 2.047105447328765
321800000 3.6948560805344837
321900000 2.770269653200867
322000000 3.1113184066273707
322100000 4.467191626541859
322200000 3.8814972517611612
322300000 2.8741582487969852
322400000 4.350829062829088
322500000 3.1816091397164583
322600000 4.072445678291569
322700000 2.692988464529298
3228

348000000 1.3655554568432438
348100000 1.3711678597206765
348200000 1.3738886377737103
348300000 1.1788433895851236
348400000 1.2599040640016417
348500000 1.3220307845359425
348600000 2.1993774512247786
348700000 2.500900598437518
348800000 0.9305762459475816
348900000 0.575227134519228
349000000 1.485856685840014
349100000 1.8978976820494353
349200000 1.2689926482715435
349300000 1.300866614076774
349400000 1.3256963124141534
349500000 1.172017736058314
349600000 1.3160801269762126
349700000 1.6248965203432917
349800000 2.2589067581831075
349900000 1.6455600385861182
350000000 1.4829269179442166
350100000 2.0605857325572807
350200000 2.5256918580799512
350300000 2.1745118899896503
350400000 1.1464108358615246
350500000 1.7931352950949178
350600000 1.2708332636520983
350700000 1.3681869257716175
350800000 1.2406369191256807
350900000 1.008510720880419
351000000 0.5314955685360743
351100000 0.3410686272233996
351200000 1.6767110255803386
351300000 1.2154810291977685
351400000 1.82925984

376000000 0.17818761266472738
376100000 0.16398871133071782
376200000 0.16701735210744872
376300000 0.16757138760196102
376400000 0.16717684820843537
376500000 0.17028519023027155
376600000 0.16355522512419024
376700000 0.1665418867490133
376800000 0.16632221302986872
376900000 0.1698879170501338
377000000 0.16881166411786083
377100000 0.16871135484990213
377200000 0.16838469718152216
377300000 0.16818566015443603
377400000 0.16739440081304244
377500000 0.16788314211358626
377600000 0.1671978030446569
377700000 0.16663795403754644
377800000 0.16656032250562247
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 0.9400717235261677
378000000 1.3142416716899952
378100000 0.9642985107649684
378200000 1.2202351230917752
378300000 1.029125263967799
378400000 1.4596181059683444
378500000 0.9234089262218773
378600000 1.2558381922785469
378700000 1.0940985088904023
378800000 0.9672385975872635
378900000 1.3142217469911126
379000000 1.0132231532659988
379100000 0.921

404100000 0.6785108553698036
404200000 0.7761561280874617
404300000 0.7373350845621368
404400000 0.782450475988384
404500000 0.7604415825364669
404600000 0.6674376186155656
404700000 0.8475002560963869
404800000 0.6698250319954906
404900000 0.7346292537485897
405000000 0.7771377137305465
405100000 0.7683984556206578
405200000 0.7482034989427228
405300000 0.6621229884075794
405400000 0.8120769750548217
405500000 0.6721025950380802
405600000 0.6609811601697763
405700000 0.7562567149652104
405800000 0.6959750357770681
405900000 0.6833375892578197
406000000 0.613737695195151
406100000 0.7268292528769057
406200000 0.6903264028428131
406300000 0.617501736034439
406400000 0.6743770599574029
406500000 0.6456052456404209
406600000 0.6232272807370562
406700000 0.5558471919806421
406800000 0.599406953480486
406900000 0.6527381125017836
407000000 0.5942580527458151
407100000 0.6117855877994326
407200000 0.7376682575661626
407300000 0.6542427231908042
407400000 0.5801611763143142
407500000 0.646641